In [6]:
# CH.15 딕셔너리 삽입 순서에 의존할 떄는 조심해라
# 과거 python3.5에서는 딕셔너리를 만들 때 순서가 보장이 되지 않았다.
# 마치 다른 언어들처럼

# python에서 이런 것들이 발생하는 이유는
# 딕셔너리 구현이 내장 hash 함수와 파이썬 인터프리터가 시작할 때 초기화되는 난수 씨앗값을 사용하는 해시 테이블 알고리즘으로 만들어졌기 때문에

baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}

print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


In [8]:
# python 3.7 이후부터는 순서대로 딕셔너리의 내용을 표시한다. (파이썬 언어 명세의 일부가 되었다.)
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(baby_names.popitem())

['cat', 'dog']
['kitten', 'puppy']
[('cat', 'kitten'), ('dog', 'puppy')]
('dog', 'puppy')


In [10]:
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'

a = MyClass()
for key, value in a.__dict__.items():
    print(f'{key} = {value}')

alligator = hatchling
elephant = calf


In [18]:
# 덕 타이핑
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 864,
}

def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks):
    return next(iter(ranks))

ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

names : ['otter', 'polar bear', 'fox']
after names : ['otter', 'fox', 'polar bear']
{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


In [3]:
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self): # 이것이 핵심일 듯
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self):
        return len(self.data)

# 아래와 같이 SortedDict()을 선언하면 인스턴스가 생성되는 것이지만
# 표준 딕셔너리의 프로토콜을 지키므로 populate_ranks 함수에서 사용할 수 있다.
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


In [5]:
# 근데 이렇게 하니 winner로 fox가 나오게 된다.
# 이유는 get_winner 함수에서 next(iter(ranks))를 사용했기 때문이다. <- 알파벳 순서 가장 먼저 오는 애가 사용

# def get_winner(ranks):
#     return next(iter(ranks))
# 딕셔너리가 어떤 특정 순서로 이터레이션된다고 가정하지 않고 구현
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


In [6]:
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('dict 인스턴스가 필요합니다.')
    return next(iter(ranks))

winner = get_winner(sorted_ranks)
print(winner)

TypeError: dict 인스턴스가 필요합니다.

In [8]:
from typing import Dict, MutableMapping

def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


In [ ]:
# 결국에 무슨 말이냐하면
# 과거 python 3.7이하에는 딕셔너리 순서가 보장이 되지 않았었다.
# 이제 python 3.7이상부터는 딕셔너리 순서가 보장이 되는데 우리가 그것을 가정하고 코드를 짜다보면 문제가 생길 수 있다.
# (덕타이핑의 문제 : 동적 타입 지정의 일종으로, 객체가 실행 시점에 어떻게 행동하는 지를 기준으로 객체의 타입을 판단하는 타입 지정 방식)
# 이 때, 만약에 알파벳 이름으로 정의된 SortedDict 클래스를 만들었다고 생각하자.
# 그렇게 되고, get_winner 함수를 쓰게 되면 내부에는 next(iter(ranks))가 있기에 알파벳 순서대로 나오게 된다.
# 이런 경우에는 get_winner 함수를 수정해주어야 한다.
# 첫번 째 방법 : 아예 items()로 for문 돌면서 rank가 1인 것을 찾는다.
# 두번 째 방법 : get_winner 함수에 isinstance를 사용해서 dict인지 확인해준다.
# 세번 째 방법 : typing 모듈을 사용해서 Dict, MutableMapping을 사용한다.